In [1]:
import torch
import pandas as pd

from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

### Tokenizer and Model

In [2]:
model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [93]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": [
      "lm_head",
      "multi_modal_p

### Data

In [94]:
DATA_DIR = Path.cwd() / "am_reasoning" / "data_files"

In [95]:
df = pd.read_csv((DATA_DIR / "pe_dataset.csv"), index_col=0)

In [111]:
df.columns

Index(['split', 'title', 'argument_bound_1', 'argument_bound_2', 'argument_id',
       'ac', 'label', 'essay'],
      dtype='object')

### Template Messages

In [ ]:
# sys_msg = """You are an expert in argumentation analysis. You will be given a complete argument. Your task is to analyze the argument holistically and classify it as exactly ONE of the following categories: Position, Claim, or Premise. 

# First, think through your analysis step by step, considering the function and structure of the argument. Explain your reasoning, identifying key characteristics that point to a specific classification.

# Then, provide your final classification as a single word: Position, Claim, or Premise."""

In [ ]:
# instruction = """### You are an expert in Argument Mining. You are given an argument component from an essay. Your task is to classify the argument component as either "MajorClaim", "Claim" or "Premise". You must return the argument component type in following JSON format:: {"component_type": [component_type (str)]}


# """

In [163]:
instruction = """### You are an expert in Argument Mining and discourse analysis. 

INPUT:
- You are given an argument component from an essay. 
- You are given the full text of the essay.

TASK:
- Your task is to classify the whole argument component as ONLY ONE of: "MajorClaim", "Claim" or "Premise".
- Explain your reasoning step-by-step before making your final classification.

Then, return your final classification in the following JSON format:
{"component_type": [component_type (str)]}

Example:

{"component_type": ["Claim"]}

"""

In [164]:
usr_msg = f"""### Essay text: {df.iloc[0]['essay']}""" + "\n\n" + f"""### Argument component: {df.iloc[0]['ac']}"""

In [165]:
print(usr_msg)

### Essay text: Should students be taught to compete or to cooperate?

It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.
First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students. What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others. During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, ho

In [166]:
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": usr_msg},
]

In [167]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device) # type: ignore

In [168]:
input_ids

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    972,   2947,    220,   2366,     20,    271,  14711,   1472,
            527,    459,   6335,    304,  14138,  26917,    323,  41602,   6492,
             13,   4815,  30521,    512,     12,   1472,    527,   2728,    459,
           5811,   3777,    505,    459,   9071,     13,    720,     12,   1472,
            527,   2728,    279,   2539,   1495,    315,    279,   9071,    382,
          66913,    512,     12,   4718,   3465,    374,    311,  49229,    279,
           4459,   5811,   3777,    439,  27785,  25002,    315,     25,    330,
          35575,  46644,    498,    330,  46644,      1,    477,    330,  42562,
           1082,  23811,     12,  83017,    701,  33811,   3094,  14656,  30308,
           1603,   3339,    701,   1620,  24790,    382,  12487,     11,    471,
            701,   1620,  24

In [169]:
print(tokenizer.decode(input_ids[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Mar 2025

### You are an expert in Argument Mining and discourse analysis. 

INPUT:
- You are given an argument component from an essay. 
- You are given the full text of the essay.

TASK:
- Your task is to classify the whole argument component as ONLY ONE of: "MajorClaim", "Claim" or "Premise".
- Explain your reasoning step-by-step before making your final classification.

Then, return your final classification in the following JSON format:
{"component_type": [component_type (str)]}

Example:

{"component_type": ["Claim"]}<|eot_id|><|start_header_id|>user<|end_header_id|>

### Essay text: Should students be taught to compete or to cooperate?

It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. Howe

In [170]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [171]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
)

In [172]:
response = outputs[0][input_ids.shape[-1]:]

In [173]:
print(tokenizer.decode(response, skip_special_tokens=True))

### Step-by-Step Explanation

1. **Understanding the Argument**: The argument starts by acknowledging the importance of competition in promoting economic development. However, it shifts its focus to the development of an individual's life, emphasizing the significance of cooperation during primary education.

2. **Identifying Key Components**: The key components of the argument are:
   - The importance of competition in promoting economic development.
   - The significance of cooperation in achieving goals and overcoming conflicts.
   - The need for cooperation in achieving success in competitions.

3. **Analyzing the Argument Structure**: The argument structure can be broken down into two main parts:
   - **Claim**: "We should attach more importance to cooperation during primary education."
   - **Premise**: "The significance of competition is that how to become more excellence to gain the victory."

4. **Classifying the Argument**: Based on the analysis, the argument can be classifie